In [1]:


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss

test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')
train = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
traine = train.copy()
teste = test.copy()
traine.head(5)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [3]:
teste.head(5)

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [4]:
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk.tokenize import sent_tokenize



# Create reference to both datasets
full_data = [traine, teste]

# --------- Winner column only for train ----------
traine['Winner'] = np.where(traine['winner_model_a'] == 1, 1,
                    np.where(traine['winner_model_b'] == 1, 2,
                    np.where(traine['winner_tie'] == 1, 3, -1)))  # -1 for unknown

# --------- Sentiment + Subjectivity + Prompt polarity ----------
for dataset in full_data:
    dataset['A_polar'] = dataset['response_a'].apply(lambda x: TextBlob(x).sentiment.polarity)
    dataset['B_polar'] = dataset['response_b'].apply(lambda x: TextBlob(x).sentiment.polarity)
    dataset['A_sub']   = dataset['response_a'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    dataset['B_sub']   = dataset['response_b'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    dataset['prompt_polar']  = np.round(dataset['prompt'].apply(lambda x: TextBlob(x).sentiment.polarity), 2)
    B_polar = np.round(dataset['B_polar'],3)
    A_polar = np.round(dataset['A_polar'], 3)
    polarity_difference = B_polar - A_polar

# --------- TF-IDF Cosine Similarity ----------
TDF = TfidfVectorizer()

# Fit on all text in both datasets
all_texts = []
for dataset in full_data:
    all_texts.extend(dataset['prompt'].tolist())
    all_texts.extend(dataset['response_a'].tolist())
    all_texts.extend(dataset['response_b'].tolist())

TDF.fit(all_texts)
del all_texts  # Save memory

# Cosine similarity loop
for dataset in full_data:
    n_samples = len(dataset)
    chunk_size = 1000
    cos_sim_a = np.zeros(n_samples)
    cos_sim_b = np.zeros(n_samples)

    for i in range(0, n_samples, chunk_size):
        end_idx = min(i + chunk_size, n_samples)
        prompt_chunk = TDF.transform(dataset['prompt'].iloc[i:end_idx])
        resp_a_chunk = TDF.transform(dataset['response_a'].iloc[i:end_idx])
        resp_b_chunk = TDF.transform(dataset['response_b'].iloc[i:end_idx])
        cos_sim_a[i:end_idx] = cosine_similarity(prompt_chunk, resp_a_chunk).diagonal()
        cos_sim_b[i:end_idx] = cosine_similarity(prompt_chunk, resp_b_chunk).diagonal()

    dataset['cos_sim_a'] = np.round(cos_sim_a, 3)
    dataset['cos_sim_b'] = np.round(cos_sim_b, 3)

# --------- Drop 'id' column if exists ----------
for dataset in full_data:
    dataset.drop(columns=['id'], errors='ignore', inplace=True)

def coherence(text):
    sentence= sent_tokenize(text)
    if len(sentence) <2:
        return 0.0
    vector  = TDF(stop_words ='english')
    try:
        transformed = vector.for_transform(sentences)
    except:
        return 0.084

Created new features to show who won between both models in one column rather than 2, this wouldd be easier to read.A feature calles Longer_response was constructed, to show which  modle had a longer repsonse, this wa screated because users might prefer to have a longer and more detailed response from the chatbot, than summarised information. we would test this by plotting sa graph to see if ther is a correlatoin.

No pattern spotted regarding length of  response and win rate. We would have to deal with the type of model beign used, more advanced models provide better answers to the  querry.

In [5]:
traine.drop(columns =['model_b'], inplace = True)
traine.drop(columns =['model_a'], inplace = True)
# Convert model names to numeric IDs for ML
for df in full_data:
    df['prompt_len_chars'] = df['prompt'].apply(lambda x: len(x))
    df['A_len'] = df['response_a'].apply(lambda x: len(x))
    df['B_len'] = df['response_b'].apply(lambda x: len(x))
    df.drop(columns =['response_b'], inplace = True)
    df.drop(columns =['response_a'], inplace = True)
    df.drop(columns =['prompt'], inplace = True)

In [6]:
traine.head(5)

,winner_model_a,winner_model_b,winner_tie,Winner,A_polar,B_polar,A_sub,B_sub,prompt_polar,cos_sim_a,cos_sim_b,prompt_len_chars,A_len,B_len
0,1,0,0,1,0.088119,0.130147,0.497048,0.490336,0.32,0.391,0.330,165,4538,1206
1,0,1,0,2,0.054014,0.148230,0.340033,0.358584,0.08,0.641,0.584,200,3114,3649
2,0,0,1,3,0.276190,0.083333,0.571429,0.197024,0.00,0.562,0.388,60,921,1835
3,1,0,0,1,0.160765,0.230551,0.607517,0.581321,0.39,0.580,0.507,87,3182,1562
4,0,1,0,2,0.203846,0.252778,0.476923,0.422222,1.00,0.731,0.803,79,1300,772


In [7]:
teste.head(5)

,A_polar,B_polar,A_sub,B_sub,prompt_polar,cos_sim_a,cos_sim_b,prompt_len_chars,A_len,B_len
0,0.000000,0.000000,0.000000,0.000000,0.50,0.684,0.788,86,31,114
1,0.077597,0.098413,0.483868,0.253175,0.03,0.260,0.457,488,1457,460
2,0.156902,0.146411,0.461835,0.497249,0.35,0.479,0.444,217,3984,3716


In [8]:
X_train = traine.drop(['winner_model_a', 'winner_model_b', 'winner_tie'], axis=1)
Y_train = traine[['winner_model_a', 'winner_model_b', 'winner_tie']].values.argmax(1)
X_train.shape,Y_train.shape,teste.shape

#from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

KF = StratifiedKFold(n_splits = 10,shuffle =True, random_state = 101)

X_test = teste.copy()
for fold,

In [9]:
from sklearn.linear_model import LogisticRegression as logreg
from sklearn.metrics import log_loss as loglos
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model =make_pipeline(StandardScaler(), logreg(multi_class ='multinomial', solver = 'lbfgs', max_iter = 10000))
model.fit(X_train,Y_train)
Y_probs = model.predict_proba(x_test)
model_order = list(getattr(model,'classes_',[0,1,2]))
target_order = [0,1,2]
final_idx = [model_order.index(c) for c in target_order]
Y_probs = Y_probs[:,final_idx]
loss = loglos(y_test,Y_probs)
acc =model.score(x_test,y_test)
score  = round(acc*100,2)


In [10]:
loss

0.000984030814755572

In [11]:
score

100.0